In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tsensor
from sklearn.preprocessing import StandardScaler

In [ ]:
np.random.seed(42)
nn = [1, 3, 4, 2, 1]
inputs = np.random.randint(10, 100, 10).reshape(10, 1)
tagets = [n + np.random.randint(-5, 5) for n in inputs]
lr = 1e-5

# instantiation ransom weights and biases
np.random.seed(42)
layers = []
for i in range(1, len(nn)):
    layers.append([np.random.rand(nn[i - 1], nn[i]) / 5 - 0.1, np.ones((1, nn[i]))])

# forward pass
l1_output = np.maximum(0, inputs @ layers[0][0] + layers[0][1])  # hidden layer 1 output
l2_output = np.maximum(
    0, l1_output @ layers[1][0] + layers[1][1]
)  # hidden layer 2 output
l3_output = np.maximum(
    0, l2_output @ layers[2][0] + layers[2][1]
)  # hidden layer 3 output
output = l3_output @ layers[3][0] + layers[3][1]  # output layers without relu

# back propagation and weight and bias updates
# from output to layer 4
output_gradient = output
l4_w_gradient = l3_output.T @ output_gradient
l4_b_gradient = np.mean(output_gradient, axis=0)
layers[3][0] -= lr * l4_w_gradient
layers[3][1] -= lr * l4_b_gradient

# from layer4 to layer3
l3_gradient = output_gradient @ layers[3][0].T * np.heaviside(l3_output, 0)
l3_w_gradient = l2_output.T @ l3_gradient
l3_b_gradient = np.mean(l3_gradient, axis=0)
layers[2][0] -= lr * l3_w_gradient
layers[2][1] -= lr * l3_b_gradient

# form layer 3 to 2
l2_gradient = l3_gradient @ layers[2][0].T * np.heaviside(l2_output, 0)
l2_w_gradient = l1_output.T @ l2_gradient
l2_b_gradient = np.mean(l2_gradient, axis=0)
layers[1][0] -= lr * l2_w_gradient
layers[1][1] -= lr * l2_b_gradient

# from 2 to 1
l1_gradient = l2_gradient @ layers[1][0].T * np.heaviside(l1_output, 0)
l1_w_gradient = inputs.T @ l1_gradient
l1_b_gradient = np.mean(l1_gradient, axis=0)
layers[0][0] -= lr * l1_w_gradient
layers[0][1] -= lr * l1_b_gradient

In [ ]:
# genrating random inputs for temperatures
def inputs_targets_genrator(n_instances: int = 10, n_features: int = 1):
    inputs = np.random.rand(n_instances, n_features)
    valid = np.random.rand(n_instances, n_features)
    targets = np.random.rand(n_instances, 1)
    valid_y = np.random.rand(n_instances, 1)
    return inputs, targets, valid, valid_y


# instantiating random weights and biases
def weights_biases(nn):
    # store the weights
    layers = []
    for i in range(1, len(nn)):
        layers.append([np.random.rand(nn[i - 1], nn[i]) / 5 - 0.1, np.ones((1, nn[i]))])
    return layers


def forwardpass(layers, batch):
    hidden_outputs = [batch.copy()]
    for i in range(len(layers)):
        batch = batch @ layers[i][0] + layers[i][1]
        if i < len(layers) - 1:
            np.maximum(batch, 0)
        hidden_outputs.append(batch.copy())
    return batch, hidden_outputs


def backpropagation(layers, hidden_outputs, grad, lr):
    for i in range(len(layers) - 1, -1, -1):
        if i != len(layers) - 1:
            grad = grad * np.heaviside(hidden_outputs[i + 1], 0)

        w_grad = hidden_outputs[i].T @ grad
        b_grad = np.mean(grad, axis=0)

        layers[i][0] -= lr * w_grad
        layers[i][1] -= lr * b_grad
        # next gradient
        grad = grad @ layers[i][0].T
    return layers


def mse(actual, predicted):
    return np.mean((actual - predicted) ** 2)


# Correct MSE gradient function
def mse_grad(actual, predicted):
    n = actual.shape[0]  # Number of samples
    return (2 / n) * (predicted - actual)

In [30]:
lr = 1e-3
epochs = 100
batch_size = 5
n_instances = 1000
n_features = 2
layer_config = [n_features, 2, 4, 3, 1]


layers = weights_biases(layer_config)

inputs, targets, valid, valid_y = inputs_targets_genrator(
    n_instances=n_instances, n_features=n_features
)

for epoch in range(epochs):
    epoch_loss = []
    for i in range(0, inputs.shape[0], batch_size):
        train_x = inputs[i : (i + batch_size)]
        train_y = targets[i : (i + batch_size)]

        prediction, hidden_ouputs = forwardpass(layers, train_x)
        loss = mse_grad(train_y, prediction)
        epoch_loss.append(np.mean(loss**2))

        layers = backpropagation(layers, hidden_ouputs, loss, lr)

    valid_pred, _ = forwardpass(layers, valid)

    print(
        f"Epoch {epoch+1}: Train MSE: {np.mean(epoch_loss)}, Valid MSE: {mse(valid_y, valid_pred)}"
    )

Epoch 1: Train MSE: 0.0252690926926085, Valid MSE: 0.10067988309449297
Epoch 2: Train MSE: 0.013362381200240882, Valid MSE: 0.08517100151940045
Epoch 3: Train MSE: 0.012796849374726754, Valid MSE: 0.0839247116741727
Epoch 4: Train MSE: 0.012778590802019672, Valid MSE: 0.08375733016766954
Epoch 5: Train MSE: 0.01277903008372786, Valid MSE: 0.0837271422042113
Epoch 6: Train MSE: 0.01277928636877364, Valid MSE: 0.08372122133464316
Epoch 7: Train MSE: 0.012779345129850789, Valid MSE: 0.08372003767269251
Epoch 8: Train MSE: 0.012779357764485228, Valid MSE: 0.08371979913633783
Epoch 9: Train MSE: 0.012779360882951323, Valid MSE: 0.08371974999635219
Epoch 10: Train MSE: 0.012779362081755723, Valid MSE: 0.08371973884168804
Epoch 11: Train MSE: 0.01277936289506358, Valid MSE: 0.08371973530622083
Epoch 12: Train MSE: 0.012779363631054643, Valid MSE: 0.08371973329899123
Epoch 13: Train MSE: 0.012779364351565809, Valid MSE: 0.08371973159828107
Epoch 14: Train MSE: 0.012779365069001307, Valid MSE: 

[2, 1]